Süper—kodu “uçtan uca” mantık sırasıyla, sade şekilde açıklayayım.
(Numaralar, dosyadaki adımlarıyla eşleşiyor.)

---

## 0) Tekrarlanabilirlik

```python
np.random.seed(42); tf.random.set_seed(42)
```

Aynı kodu her çalıştırdığında aynı sonuçları alman için rastgelelik sabitleniyor.

---

## 1) Veri üretimi (sentetik zaman serisi)

```python
series = 10 + trend + season + noise
```

* **trend:** zamanla hafif artış
* **season:** sinüs dalgası (mevsimsellik)
* **noise:** gürültü
  Gerçeğe benzeyen bir tek değişkenli seri üretiyoruz.

---

## 2) Train / Validation / Test bölme

```python
70% train, 15% val, 15% test
```

Modeli **öğretmek**, **ayarlamak** ve **sınamak** için üçe ayırıyoruz.

---

## 3) Ölçekleme (MinMaxScaler)

```python
scaler.fit(train); transform(val, test)
```

* Yalnız **train** ile “fit” edilir (bilgi sızdırmamak için).
* Tüm veriler 0–1 aralığına çekilir → LSTM’nin öğrenmesi hızlanır/stabilleşir.

---

## 4) Pencereleme (windowing)

```python
LOOKBACK = 30  # geçmiş 30 gün girdi
HORIZON  = 1   # 1 gün ileri tahmin
X:(num_samples, 30, 1), y:(num_samples, 1)
```

Zaman serisini “kaydırmalı pencereler”e çevirir:

* Örn: lookback=3 için
  `[v0,v1,v2] -> hedef v3`
  `[v1,v2,v3] -> hedef v4` …
  Böylece LSTM’e 30 günlük blok verir, “yarını” isteriz.

---

## 5) Model mimarisi

```python
LSTM(64, return_sequences=True) -> Dropout(0.2)
LSTM(32) -> Dropout(0.2)
Dense(horizon)
```

* İlk LSTM **return\_sequences=True**: çıktıyı adım adım verir ki ikinci LSTM tüm sekansı işleyebilsin.
* **Dropout:** aşırı öğrenmeyi azaltır.
* **Dense(horizon):** son adımda sayısal tahmin (regresyon).

**Compile:**

```python
optimizer=Adam(1e-3), loss="mse", metric="mae"
```

* Regresyonda **MSE** iyi bir kayıp; **MAE** anlaşılır bir hata metriği.

---

## 6) Callbacks (akıllı eğitim)

* **EarlyStopping(patience=10):** Val. loss 10 epoch düzelmezse durur, en iyi ağı geri yükler.
* **ReduceLROnPlateau(patience=5):** İyileşme durursa lr’ı 0.5’e düşürür.
* **ModelCheckpoint:** En iyi modeli diske kaydeder.

---

## 7) Eğitim

```python
epochs=100, batch_size=64
```

Mini-batch eğitim yapılır; callback’ler sayesinde genelde 100’e varmadan durur.

---

## 8) En iyi modeli yükleme

```python
load_model("best_lstm.keras")
```

Checkpoint’teki en başarılı ağı kullanırız.

---

## 9) Testte tahmin + ölçeği geri alma

```python
pred_test_scaled = model.predict(X_test)
y_test_inv, pred_test_inv = inverse_scale(...)
```

* Tahminler 0–1 aralığında; raporlamak için **inverse\_transform** ile orijinal ölçeğe çevrilir.
* **Neden küçük bir yardımcı fonksiyon var?** `MinMaxScaler` 2D beklediği için (N,1) şekline getirip geri çeviriyoruz.

**Metrikler:**

```python
MSE, RMSE, MAE
```

Test performansını sayısal olarak verir.

---

## 10) Grafikler

* **Eğitim eğrileri:** train/val loss → aşırı öğrenme var mı gör.
* **Test karşılaştırma:** Gerçek vs Tahmin eğrileri → modelin izleme kabiliyeti.

---

## 11) Tek örnek tahmin

Son test penceresiyle tek bir “yarın” tahmini basılır (hızlı sanity check).

---

# Kısa Özet (zihinsel model)

1. **Veri hazırla** (böl, ölçekle) →
2. **Pencere oluştur** (lookback→horizon) →
3. **LSTM modeli kur** (2 katman + dropout) →
4. **Akıllı eğitim** (early stop, lr azaltma, checkpoint) →
5. **Test et** (inverse scale + MSE/MAE) →
6. **Grafiklerle kontrol et**.

---

## Sık yapılan ayarlar

* **Daha kısa/uzun hafıza:** `LOOKBACK` (örn. 14/60/120).
* **Çok-adımlı tahmin:** `HORIZON>1` ve `Dense(HORIZON)`.
* **GRU dene:** `layers.LSTM` → `layers.GRU`.
* **Çok değişkenli seri:** `features>1` (X’e ek sütunlar).
* **Düzenleme:** Dropout oranı, LSTM nöron sayıları, öğrenme oranı.

Hazırsan bunu GRU varyantıyla veya çok değişkenli (multivariate) versiyonla da gösterebilirim.
